In [2]:
import numpy as np
import pandas as pd
from scipy import stats

# T-검정
## 일표본 T-검정

In [3]:
data1 = stats.norm.rvs(loc=10, scale=2, size=500, random_state = 2022)
data2 = stats.t.rvs(loc=10, scale=2, df=3, size=500, random_state=2022)
print(f'평균 : {data1.mean()}, 분산 : {data1.var()}')
print(f'평균 : {data2.mean()}, 분산 : {data2.var()}')

평균 : 10.004707240081995, 분산 : 3.997456829579718
평균 : 10.055494229069078, 분산 : 8.99130725100136


귀무가설 : 모평균의 값은 $u_0$이다.<br>
t-test를 수행하기 위해서는 정규성을 먼저 점검해야한다.

In [4]:
# 정규성 입증
_,p_val1 = stats.shapiro(data1)
_,p_val2 = stats.shapiro(data2)
print(f'p_value of shapiro-wilk test with data1 : {p_val1},\np_value of shapiro-wilk test with data2 : {p_val2}')

p_value of shapiro-wilk test with data1 : 0.5016745924949646,
p_value of shapiro-wilk test with data2 : 3.5778968153553185e-15


데이터가 정규분포를 따른다는 가정을 만족하는 경우 t-test, 정규성을 만족하지 못할 경우 wilcoxon test를 이용해 가설을 검정할 수 있다.

In [5]:
print(stats.ttest_1samp(data1,10))
print(stats.wilcoxon(data2-10))

Ttest_1sampResult(statistic=0.05259261081891289, pvalue=0.9580775517179503)
WilcoxonResult(statistic=62011.0, pvalue=0.8493435848163181)


## 대응표본 T-검정
단일모집단에 대해 어떤 행동을 수행한 경우 그 행동 수행에 따른 평균 차이를 비교, 표본의 크기가 같음<br>
ex) 어떤 실험군에 대해 혈압약의 투약 효과를 측정하고자 투약 이전과 이후의 혈압 수치를 비교할 수 있음

귀무가설 : 두 데이터셋의 모평균간 차이가 없다.

In [6]:
# 수면제 복용 전과 후의 수면시간
before = np.array([7,3,4,5,2,1,6,6,5,4])
after = np.array([8,4,5,6,2,3,6,8,6,5])
# ttest_rel은 related
# less는 before < after라는 뜻
stats.ttest_rel(before, after,alternative='less')

Ttest_relResult(statistic=-4.743416490252569, pvalue=0.0005269356285082765)

In [7]:
stats.ttest_rel(before, after,alternative='two-sided')

Ttest_relResult(statistic=-4.743416490252569, pvalue=0.001053871257016553)

In [8]:
stats.ttest_rel(before, after,alternative='greater')

Ttest_relResult(statistic=-4.743416490252569, pvalue=0.9994730643714917)

In [13]:
stats.levene(before, after)

LeveneResult(statistic=0.0, pvalue=1.0)

## 독립표본 T-검정
귀무가설 : 두 데이터셋의 모평균간 차이가 없다.<br>
등분산성에 따라 절차가 다르기 때문에 미리 등분산 검정을 실시해야한다.

In [18]:
cats = pd.read_csv('datasets/cats.csv')
cats = cats.drop('Unnamed: 0',axis=1)
cats.head()

,Sex,Bwt,Hwt
0,F,2.0,7.0
1,F,2.0,7.4
2,F,2.0,9.5
3,F,2.1,7.2
4,F,2.1,7.3


In [19]:
cats_female = cats[cats['Sex']=='F']
cats_male = cats[cats['Sex']=='M']

In [21]:
# 정규성을 만족하지 않으므로 Levene 테스트 진행
print(stats.shapiro(cats_male['Bwt']))
print(stats.shapiro(cats_female['Bwt']))

ShapiroResult(statistic=0.9788321852684021, pvalue=0.11895745247602463)
ShapiroResult(statistic=0.8909613490104675, pvalue=0.0003754299250431359)


In [25]:
# 등분산성을 만족하지 않음
stats.levene(cats_male['Bwt'], cats_female['Bwt'])

LeveneResult(statistic=19.43101190877999, pvalue=2.0435285255189404e-05)

In [27]:
# 등분산에 관한 인자를 넣어준다.

# 귀무가설을 기각하고 '고양이 성별에 따른 몸무게에는 통계적으로 유의한 차이가 존재한다' 결론 지을 수 있다.
stats.ttest_ind(cats_male['Bwt'], cats_female['Bwt'],equal_var=False)

Ttest_indResult(statistic=8.70948849909559, pvalue=8.831034455859356e-15)

In [32]:
cats.groupby('Sex').mean()['Bwt']

Sex
F    2.359574
M    2.900000
Name: Bwt, dtype: float64

In [33]:
stats.ttest_ind(cats_male['Bwt'], cats_female['Bwt'],equal_var=False,alternative='greater')

Ttest_indResult(statistic=8.70948849909559, pvalue=4.415517227929678e-15)

# 교차분석(with chi-square distribution)
## 적합성 검정(동질성 검정)
실험에서 얻어진 관측값과 이론적으로 예상한 값들이 일치하는지를 검정<br>
귀무가설 : 실제 분포와 이론적 분포간 차이가 없다.<br>
대립가설 : 두 분포가 일치하지 않는다.

In [36]:
survey = pd.read_csv('datasets/survey.csv',index_col=0)

In [38]:
survey.head()

,Sex,Wr.Hnd,NW.Hnd,W.Hnd,Fold,Pulse,Clap,Exer,Smoke,Height,M.I,Age
1,Female,18.5,18.0,Right,R on L,92.0,Left,Some,Never,173.0,Metric,18.250
2,Male,19.5,20.5,Left,R on L,104.0,Left,None,Regul,177.8,Imperial,17.583
3,Male,18.0,13.3,Right,L on R,87.0,Neither,None,Occas,NaN,NaN,16.917
4,Male,18.8,18.9,Right,R on L,NaN,Neither,None,Never,160.0,Metric,20.333
5,Male,20.0,20.0,Right,Neither,35.0,Right,Some,Never,165.0,Metric,23.667


R의 chisq.test와는 다른 제약사항으로 f_obs의 합과 f_exp의 합이 같아야함<br>
$\textbf{f_obs의 값은 그대로 두고, f_exp를 비율에 맞게 바꿔줄 것}$

다음 예시는 왼손잡이와 오른손잡이의 비율이 2:8이 맞는지 검정하는 것

In [80]:
observed = survey['W.Hnd'].value_counts()
stats.chisquare(f_obs = observed.sort_values(),
                f_exp = np.array([0.2,0.8])*observed.sum())

Power_divergenceResult(statistic=22.58050847457627, pvalue=2.0151876969887844e-06)